In [1]:
import pandas as pd
import numpy as np
from vnstock import Vnstock

In [3]:
# Load danh sách mã cổ phiếu từ file Excel
excel_file = r"C:\Users\User\Downloads\merged_stocks.xlsx"
symbol_df = pd.read_excel(excel_file)

# Chuyển đổi cột "Mã" thành danh sách mã chứng khoán
symbol_list = symbol_df["Mã"].tolist()

In [45]:
# Hàm lấy dữ liệu chứng khoán từ Vnstock
def get_stock_data(symbol, start_date="2020-01-01", end_date="2024-12-31"):
    stock = Vnstock().stock(symbol=symbol, source="VCI")
    df = stock.quote.history(start=start_date, end=end_date, interval="1D")
    if df is not None and not df.empty:
        df["time"] = pd.to_datetime(df["time"])  # Sửa từ "date" thành "time"
        return df, stock
    return None, None


In [105]:

# Hàm lấy dữ liệu tài chính từ finance.ratio()
def get_finance_data(stock, symbol, date):
    finance_data = stock.finance.ratio()
    if finance_data.empty:
        return np.nan, np.nan, np.nan
    
    year = date.year
    quarter = (date.month - 1) // 3 + 1  # Xác định quý từ tháng
    
    finance_filtered = finance_data[
        (finance_data[("Meta", "ticker")] == symbol) &
        (finance_data[("Meta", "yearReport")] == year) &
        (finance_data[("Meta", "lengthReport")] == quarter)
    ]
    
    if finance_filtered.empty:
        return np.nan, np.nan, np.nan
    
    shares_outstanding = finance_filtered.get(("Chỉ tiêu định giá", "Outstanding Share (Mil. Shares)"), np.nan).values[-1]
    pe_ttm = finance_filtered.get(("Chỉ tiêu định giá", "P/E"), np.nan).values[-1]
    dividend_yield = finance_filtered.get(("Chỉ tiêu khả năng sinh lợi", "Dividend yield (%)"), np.nan).values[-1]
    
    return shares_outstanding, pe_ttm, dividend_yield


In [ ]:
# Hàm tính toán các chỉ số chứng khoán
def calculate_stock_metrics(symbol, date):
    df, stock = get_stock_data(symbol)
    if df is None or df.empty:
        return None

    df = df[df["time"] <= date]
    if df.empty:
        return None
    
    shares_outstanding, pe_ttm, dividend_yield = get_finance_data(stock, symbol, date)
    
    df_52wk = df[df["time"] >= (date - pd.Timedelta(days=365))]
    high_52wk = df_52wk["high"].max() if not df_52wk.empty else np.nan
    low_52wk = df_52wk["low"].min() if not df_52wk.empty else np.nan
    ratio_52wk = high_52wk / low_52wk if low_52wk > 0 else np.nan
    
    avg_vol_5d = df["volume"].rolling(window=5, min_periods=1).mean().iloc[-1]
    avg_vol_10d = df["volume"].rolling(window=10, min_periods=1).mean().iloc[-1]
    
    df["returns"] = df["close"].pct_change()
    beta_value = df["returns"].cov(df["returns"]) / df["returns"].var() if df["returns"].var() != 0 else np.nan
    
    def percentage_change(days):
        if len(df) < days:
            return np.nan
        return ((df["close"].iloc[-1] - df["close"].iloc[-days]) / df["close"].iloc[-days]) * 100
    
    percent_change = {
        "1 Day": percentage_change(1),
        "5 Day": percentage_change(5),
        "3 Months": percentage_change(63),
        "6 Months": percentage_change(126),
        "Month to Date": percentage_change(df[df["time"].dt.month == date.month].shape[0]),
        "Year to Date": percentage_change(df[df["time"].dt.year == date.year].shape[0]),
    }
    
     # ANALYST OUTLOOK
    analyst_ratings = {
        "Strong Buy": np.random.randint(0, 5),
        "Buy": np.random.randint(0, 5),
        "Hold": np.random.randint(0, 5),
        "Sell": np.random.randint(0, 5),
        "Strong Sell": np.random.randint(0, 5),
    }
    total_votes = sum(analyst_ratings.values())
    recommendation_score = (analyst_ratings["Strong Buy"] * 1 +
                            analyst_ratings["Buy"] * 2 +
                            analyst_ratings["Hold"] * 3 +
                            analyst_ratings["Sell"] * 4 +
                            analyst_ratings["Strong Sell"] * 5) / total_votes if total_votes > 0 else np.nan
    recommendation = "Strong Buy" if recommendation_score < 1.5 else "Buy" if recommendation_score < 2.5 else "Hold" if recommendation_score < 3.5 else "Sell" if recommendation_score < 4.5 else "Strong Sell"

    return {
        "Symbol": symbol,
        "Date": date.strftime("%Y-%m-%d"),
        "SHARE DETAIL": {
            "52 Wk High / Low": ratio_52wk,
            "5-day Avg Volume": avg_vol_5d,
            "10-day Avg Volume": avg_vol_10d,
            "Beta Value": beta_value,
            "Shares Outstanding": shares_outstanding
        },
        "RATIO": {
            "P/E (TTM)": pe_ttm,
            "Dividend Yield (%)": dividend_yield
        },
        "Percentage Change": percent_change,
        "ANALYST OUTLOOK": {
            "Ratings": analyst_ratings,
            "Recommendation Score": recommendation_score,
            "Recommendation": recommendation
        }
    }

In [118]:
# Hàm tra cứu một mã chứng khoán vào một ngày cụ thể
def lookup_stock(symbol, date):
    date = pd.to_datetime(date)
    if symbol not in symbol_list:
        print(f" Mã chứng khoán {symbol} không tồn tại trong danh sách!")
        return None
    metrics = calculate_stock_metrics(symbol, date)
    if metrics:
        print(f"Thông tin chứng khoán:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
    else:
        print(f"Không tìm thấy dữ liệu cho ngày này.")
    return metrics

In [120]:
# Chạy thử với một mã cổ phiếu
lookup_stock("ACB", "2022-12-31")

2025-03-08 16:27:31 - vnstock.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Thông tin chứng khoán:
Symbol: ACB
Date: 2022-12-31
SHARE DETAIL: {'52 Wk High / Low': 1.6666666666666665, '5-day Avg Volume': 1760118.6, '10-day Avg Volume': 2033972.0, 'Beta Value': 0.9999999999999984, 'Shares Outstanding': 3377435094.0}
RATIO: {'P/E (TTM)': 6.1561819167, 'Dividend Yield (%)': 0.0400801603}
Percentage Change: {'1 Day': 0.0, '5 Day': -0.9026434558349488, '3 Months': 5.346127484578474, '6 Months': -9.481743227326273, 'Month to Date': 5.346127484578474, 'Year to Date': -20.40393578456758}
ANALYST OUTLOOK: {'Ratings': {'Strong Buy': 2, 'Buy': 1, 'Hold': 1, 'Sell': 3, 'Strong Sell': 3}, 'Recommendation Score': 3.4, 'Recommendation': 'Hold'}


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


{'Symbol': 'ACB',
 'Date': '2022-12-31',
 'SHARE DETAIL': {'52 Wk High / Low': 1.6666666666666665,
  '5-day Avg Volume': 1760118.6,
  '10-day Avg Volume': 2033972.0,
  'Beta Value': 0.9999999999999984,
  'Shares Outstanding': 3377435094.0},
 'RATIO': {'P/E (TTM)': 6.1561819167, 'Dividend Yield (%)': 0.0400801603},
 'Percentage Change': {'1 Day': 0.0,
  '5 Day': -0.9026434558349488,
  '3 Months': 5.346127484578474,
  '6 Months': -9.481743227326273,
  'Month to Date': 5.346127484578474,
  'Year to Date': -20.40393578456758},
 'ANALYST OUTLOOK': {'Ratings': {'Strong Buy': 2,
   'Buy': 1,
   'Hold': 1,
   'Sell': 3,
   'Strong Sell': 3},
  'Recommendation Score': 3.4,
  'Recommendation': 'Hold'}}